In [1]:
import os
import numpy as np
import pandas as pd

In [30]:
DATA_PATH = './sample_data/'

In [33]:
t = df_read(DATA_PATH + 'BTCUSD/BTCUSD-2017-05.h5')
t.head()

,ask,bid,ask_volume,bid_volume
date,,,,
2017-05-07 23:59:59.058,1558.6,1555.1,17000,14000
2017-05-07 23:59:59.188,1558.2,1555.2,13000,25000
2017-05-07 23:59:59.324,1557.9,1555.0,9800,4800
2017-05-07 23:59:59.368,1558.2,1555.0,23000,12000
2017-05-07 23:59:59.494,1558.7,1554.7,10000,4200


In [34]:
df = df_load(DATA_PATH, 'BTCUSD', '2017-05', '2020-04')

FileNotFound: ./sample_data/BTCUSD/BTCUSD-2017-06.h5
FileNotFound: ./sample_data/BTCUSD/BTCUSD-2017-07.h5
FileNotFound: ./sample_data/BTCUSD/BTCUSD-2017-08.h5
FileNotFound: ./sample_data/BTCUSD/BTCUSD-2017-09.h5
FileNotFound: ./sample_data/BTCUSD/BTCUSD-2017-10.h5
FileNotFound: ./sample_data/BTCUSD/BTCUSD-2017-11.h5
FileNotFound: ./sample_data/BTCUSD/BTCUSD-2017-12.h5
FileNotFound: ./sample_data/BTCUSD/BTCUSD-2018-01.h5
FileNotFound: ./sample_data/BTCUSD/BTCUSD-2018-02.h5
FileNotFound: ./sample_data/BTCUSD/BTCUSD-2018-03.h5
FileNotFound: ./sample_data/BTCUSD/BTCUSD-2018-04.h5
FileNotFound: ./sample_data/BTCUSD/BTCUSD-2018-05.h5
FileNotFound: ./sample_data/BTCUSD/BTCUSD-2018-06.h5
FileNotFound: ./sample_data/BTCUSD/BTCUSD-2018-07.h5
FileNotFound: ./sample_data/BTCUSD/BTCUSD-2018-08.h5
FileNotFound: ./sample_data/BTCUSD/BTCUSD-2018-09.h5
FileNotFound: ./sample_data/BTCUSD/BTCUSD-2018-10.h5
FileNotFound: ./sample_data/BTCUSD/BTCUSD-2018-11.h5
FileNotFound: ./sample_data/BTCUSD/BTCUSD-2018

In [35]:
df

,ask,bid,ask_volume,bid_volume
date,,,,
2017-05-07 23:59:59.058,1558.6,1555.1,17000,14000
2017-05-07 23:59:59.188,1558.2,1555.2,13000,25000
2017-05-07 23:59:59.324,1557.9,1555.0,9800,4800
2017-05-07 23:59:59.368,1558.2,1555.0,23000,12000
2017-05-07 23:59:59.494,1558.7,1554.7,10000,4200
...,...,...,...,...
2020-03-31 23:59:48.667,6481.3,6369.4,22,7500
2020-03-31 23:59:55.796,6481.2,6369.4,22,7500
2020-03-31 23:59:56.018,6481.1,6369.4,22,7500


### loader.py

In [36]:
def df_read(path):
    try:
        df = pd.read_hdf(path)
    except FileNotFoundError:
        path = os.path.splitext(path)[0] + '.pkl'
        df = pd.read_pickle(path)
    df.reset_index(drop=True, inplace=True)
    df.set_index('date', inplace=True)
    return df

In [37]:
def df_load(folder, symbol, start, end):
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)
    if end.day > 1 or end > end.floor('d'):
        range_end = end.floor('d') + pd.offsets.MonthBegin(1)
    else:
        range_end = end
    data_range = pd.date_range(start=start, end=range_end, freq='M')
    filenames = [os.path.join(folder, symbol, f'{symbol}-{d.year}-{d.month:02}.h5') for d in data_range]
    dframes = []
    for f in filenames:
        try:
            df = df_read(f)
            dframes.append(df)
        except FileNotFoundError:
            print('FileNotFound:', f)
    df = pd.concat(dframes)
    df = df.loc[start:end]
    # Pandas df.loc[start_date : end_date] includes both end-points in the result
    if df.index[-1] == end:
        df.drop(df.index[-1], inplace=True)
    return df

### bars.py

In [59]:
ndf = get_timebars(df, '3T')

In [60]:
ndf

ask                             bid                  \
                        max     min   first    last     max     min   first   
date                                                                          
2017-05-07 23:57:00  1591.3  1557.9  1558.6  1591.3  1590.2  1554.4  1555.1   
2017-05-08 00:00:00  1601.8  1557.6  1557.9  1592.4  1600.7  1554.3  1555.1   
2017-05-08 00:03:00  1594.1  1556.9  1592.1  1562.3  1591.1  1551.9  1591.1   
2017-05-08 00:06:00  1602.0  1561.6  1562.3  1592.6  1600.6  1556.7  1557.5   
2017-05-08 00:09:00  1601.6  1562.9  1592.3  1592.3  1600.6  1559.7  1591.1   
...                     ...     ...     ...     ...     ...     ...     ...   
2020-03-31 23:45:00  6501.4  6493.5  6495.0  6496.2  6389.3  6379.4  6382.8   
2020-03-31 23:48:00  6503.0  6496.0  6496.2  6503.0  6391.2  6382.7  6383.2   
2020-03-31 23:51:00  6503.0  6497.0  6503.0  6498.7  6391.3  6384.8  6391.2   
2020-03-31 23:54:00  6502.2  6495.1  6498.7  6495.1  6393.3  6380.6  6389.2   
2020-03-31 23:57:00  6495.2  6479.6  6495.2  6481.2  6381.0  6366.0  6380.6   

                               spr                         ask_volume  \
                       last    max    min  first   last           sum   
date                                                                    
2017-05-07 23:57:00  1589.9    4.0    0.8    3.5    1.4  1.212000e-07   
2017-05-08 00:00:00  1591.1   12.5    0.2    2.8    1.3  7.255400e-06   
2017-05-08 00:03:00  1557.2   42.2    0.0    1.0    5.1  1.509500e-06   
2017-05-08 00:06:00  1591.1   10.1    0.5    4.8    1.5  1.007170e-05   
2017-05-08 00:09:00  1591.6    9.7    0.0    1.2    0.7  3.508800e-06   
...                     ...    ...    ...    ...    ...           ...   
2020-03-31 23:45:00  6383.0  115.0  110.2  112.2  113.2  2.202800e-07   
2020-03-31 23:48:00  6390.8  114.3  110.3  113.0  112.2  4.202000e-09   
2020-03-31 23:51:00  6389.1  115.0  108.4  111.8  109.6  4.352000e-08   
2020-03-31 23:54:00  6380.6  115.0  108.4  109.5  114.5  5.542600e-08   
2020-03-31 23:57:00  6370.4  115.0  110.2  114.6  110.8  1.144220e-07   

                       bid_volume ticks  
                              sum count  
date                                     
2017-05-07 23:57:00  1.280000e-07     9  
2017-05-08 00:00:00  7.241000e-06   520  
2017-05-08 00:03:00  1.600800e-06   114  
2017-05-08 00:06:00  1.016980e-05   737  
2017-05-08 00:09:00  3.757400e-06   265  
...                           ...   ...  
2020-03-31 23:45:00  1.818098e-06   268  
2020-03-31 23:48:00  1.432500e-06   191  
2020-03-31 23:51:00  1.217569e-06   165  
2020-03-31 23:54:00  1.020069e-06   139  
2020-03-31 23:57:00  1.532753e-06   215  

[17491 rows x 15 columns]

In [58]:
pd.Timedelta('1s')

Timedelta('0 days 00:00:01')

In [38]:
def get_timebars(df, time_frame, ffill=True, active=True, show_intervals=False):
    # Get spread
    df['spr'] = df['ask'] - df['bid']
    df['ask_volume'] /= 1000
    df['bid_volume'] /= 1000

    agg = {
        'ask':['max', 'min', 'first', 'last'],
        'bid':['max', 'min', 'first', 'last'],
        'spr':['max', 'min', 'first', 'last'],
        'ask_volume':['sum'],
        'bid_volume':['sum']
    }

    # Resample ticks
    resampled = df.resample(time_frame)
    ndf = resampled.agg(agg)
    ndf[('ticks', 'count')] = resampled['ask'].count()

    # Get trading hours (with more than one tick in half hour).
    assert pd.Timedelta('30 min') % pd.Timedelta(time_frame) == pd.Timedelta(0)
    open = (ndf[('ticks', 'count')]
        .resample('30 min')
        .sum()
        .reindex(ndf.index, method='ffill')) > 0

    if active:
        # keep only open hours
        ndf = ndf[open]
    else:
        ndf['open'] = open

    if show_intervals:
        print("Sessions intervals:")
        print(ndf.index[open != open.shift()])

    if ffill:
        # Forward fill empty rows (no ticks)
        ndf = ndf.ffill()
        # Recompute 'max', 'min', and 'first' of ffill-ed rows (with no ticks)
        row0 = ndf[('ticks', 'count')] == 0
        ndf.loc[row0, [('ask','max'), ('ask','min'), ('ask','first')]] = ndf.loc[row0, ('ask','last')]
        ndf.loc[row0, [('bid','max'), ('bid','min'), ('bid','first')]] = ndf.loc[row0, ('bid','last')]
        ndf.loc[row0, [('ask_volume','sum'), ('bid_volume','sum')]] = 0
        # remove 
    return ndf

In [39]:
def get_tickbars(df, nticks):
    # Keep datetime index as a column
    df = df.reset_index()

    # Get spread
    df['spr'] = df['ask'] - df['bid']
    df['ask_volume'] /= 1000
    df['bid_volume'] /= 1000

    agg = {
        'ask':['max', 'min', 'first', 'last'],
        'bid':['max', 'min', 'first', 'last'],
        'spr':['max', 'min', 'first', 'last'],
        'date':['first', 'last'],
        'ask_volume':['sum'],
        'bid_volume':['sum']
    }

    # Groupby ticks
    resampled = df.groupby(np.arange(len(df.index)) // nticks)
    ndf = resampled.agg(agg)
    ndf[('date', 'delta')] = ndf[('date', 'last')].diff()
    ndf.loc[0, ('date', 'delta')] = ndf.loc[0, ('date', 'last')] - ndf.loc[0, ('date', 'first')]

    return ndf

In [40]:
def add_rolling(df, window, columns, function):

    new_columns = [f'{c}_{window}' for c in columns]
    new_multi_index = pd.MultiIndex.from_product([new_columns, [function]])

    if function == 'first':
        values = df[pd.MultiIndex.from_product([columns, [function]])]
        df[new_multi_index] = values.shift(window) # .fillna(values.iloc[0])
    elif function == 'mean':
        values = df[pd.MultiIndex.from_product([columns, ['last']])]
        df[new_multi_index] = values.rolling(window).mean()
    else:
        values = df[pd.MultiIndex.from_product([columns, [function]])]
        rolling = values.rolling(window)
        method = getattr(rolling, function)
        df[new_multi_index] = method()

In [41]:
def add_rolling_bars(df, tf):
    
    # Basic features
    columns = ['ask', 'bid', 'spr']
    for feature in ['max', 'min', 'first', 'mean']:
        add_rolling(df, tf, columns, feature)

    # Addtional features
    if pd.api.types.is_integer_dtype(df.index):
        add_rolling(df, tf, ['date'], 'first')
        df[(f'date_{tf}', 'delta')] = df[('date', 'last')] - df[('date', 'last')].shift(tf)
        columns = [('ask_volume', 'sum'), ('bid_volume', 'sum')]
        newcols = [(f'ask_volume_{tf}', 'mean'), (f'bid_volume_{tf}', 'mean')]
    else:
        columns = [('ask_volume', 'sum'), ('bid_volume', 'sum'), ('ticks', 'count')]
        newcols = [(f'ask_volume_{tf}', 'mean'), (f'bid_volume_{tf}', 'mean'), (f'ticks_{tf}', 'mean')]

    df[newcols] = df[columns].rolling(tf).mean()

    return df